<a href="https://colab.research.google.com/github/lclazx/nlp_learning/blob/master/classify_text_with_tf_hub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 引入资源和设计辅助类、函数

In [0]:
import tensorflow as tf
import tensorflow_hub as hub

In [34]:
tf.__version__

'1.15.0'

In [0]:
class InputFeature(object):
  def __init__(self,
         input_ids,
         input_mask,
         segment_ids,
         label_ids,
         is_real_example=True):
    self.input_ids = input_ids
    self.input_mask = input_mask
    self.segment_ids = segment_ids
    self.label_ids = label_ids
    self.is_real_example = is_real_example


In [0]:
def input_fn_builder(features, seq_length, is_training, num_labels, drop_remainder):
  all_input_ids = []
  all_input_mask = []
  all_segment_ids = []
  all_label_ids = []

  for feature in features:
    all_input_ids.append(feature.input_ids)
    all_input_mask.append(feature.input_mask)
    all_segment_ids.append(feature.segment_ids)
    all_label_ids.append(feature.label_ids)

  def input_fn(params):
    batch_size = params['batch_size']
    num_examples = len(features)
    d = tf.data.Dataset.from_tensor_slices({
        'input_ids': 
          tf.constant(
              all_input_ids, shape=[num_examples, seq_length],
              dtype=tf.int32),
        'input_mask':
          tf.constant(all_input_mask, shape=[num_examples, seq_length], dtype=tf.int32),
        'segment_ids':
          tf.constant(all_segment_ids, shape=[num_examples, seq_length], dtype=tf.int32),
        'label_ids':
          tf.constant(all_label_ids, shape=[num_examples, num_labels], dtype=tf.int32)
    })

    if is_training:
      d = d.repeat()
      d = d.shuffle(buffer_size=100)
    d = d.batch(batch_size=batch_size, drop_remainder=drop_remainder)
    return d
  return input_fn

In [0]:
def convert_single_example(ex_index, example, label_list, max_seq_length, tokenizer):
  label_map = {}
  # for(i, label) in enumrate(label_list):
  #   label_map[label] = id
  tokens_a = tokenizer.tokenize(example.text_a)
  tokens_b = None

  if len(tokens_a) > max_seq_length - 2:
    tokens_a = tokens_a[0: (max_seq_length - 2)]
  tokens = []
  segment_ids = []
  tokens.append('[CLS]')
  segment_ids.append(0)
  for token in tokens_a:
    tokens.append(token)
    segment_ids.append(0)
  tokens.append('[SEP]')
  segment_ids.append(0)

  input_ids = tokenizer.convert_tokens_to_ids(tokens)
  input_mask = [1] * len(input_ids)

  while len(input_ids) < max_seq_length:
    input_ids.append(0)
    input_mask.append(0)
    segment_ids.append(0)

  assert len(input_ids) == max_seq_length
  assert len(input_mask) == max_seq_length
  assert len(segment_ids) == max_seq_length

  label_ids = [0] * len(label_list)
  for label in example.label:
    label_ids[label] = 1

  feature = InputFeature(input_ids=input_ids, input_mask = input_mask, segment_ids = segment_ids, label_ids = label_ids, is_real_example=True)
  return feature

def conver_examples_to_features(examples, label_list, max_seq_length, tokenizer):
  features=[]
  for(ex_index, example) in enumerate(examples):
    if ex_index%10000 == 0:
      tf.logging.info("Writing example %d of %d" % (ex_index, len(examples)))
    feature = convert_single_example(ex_index, example, label_list, max_seq_length, tokenizer)
    features.append(feature)
  return features

    

# 步骤1


## 引用 google cloud service 包， 并下载文件

In [38]:
from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
gcs_service = build('storage', 'v1')

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.locked_file'

During handling of the above exception, another exceptio

## 创建输出目录

In [0]:

OUTPUT_DIR = 'training_output1' #@param
DO_DELETE =  False#@param
USE_BUCKET = True #@param
BUCKET = 'bert_classification'

if USE_BUCKET:
  OUTPUT_DIR = 'gs://{}/{}'.format(BUCKET, OUTPUT_DIR)
  
if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    pass
tf.gfile.MakeDirs(OUTPUT_DIR)
  

## 下载文件

In [40]:
from apiclient.http import MediaIoBaseDownload
def download_file(output_dir, source_file):
  with open(output_dir, 'wb') as f:
    request = gcs_service.objects().get_media(bucket=BUCKET, object=source_file)
    media = MediaIoBaseDownload(f, request)
    done = False
    while not done:
      _, done = media.next_chunk()

download_file(output_dir='/tmp/train_data.json', source_file='raw_data/train_data.json/train_data.json')
download_file(output_dir='/tmp/test_data.json', source_file='raw_data/test_data_postag.json/test_data_postag.json')
download_file(output_dir='/tmp/dev_data.json', source_file='raw_data/dev_data.json/dev_data.json')

print('Downloaded')

Downloaded


# 步骤2


## 准备样本

In [0]:
import json
import pandas as pd
def prepare_data(input_file, is_training=True, predicate_list=None, limit=None):
  if not predicate_list:
    predicate_list = []
  data = {
      'text':[],
      'labels':[]      
  }
  with open(input_file, 'r') as f:
    for (index, line) in enumerate(f):
      if limit and index>=limit:
        break
      if index % 10000 == 0:
        print('sample {}', index)
      line_data = json.loads(line)
      text = line_data['text']
      data['text'].append(text)
      labels = []
      for spo in line_data['spo_list']:
        predicate = spo['predicate']
        if is_training:
          if predicate_list.count(predicate) == 0:
            predicate_list.append(predicate)
        else:
          if predicate_list.count(predicate) == 0:
            continue

        predicate_index = predicate_list.index(predicate)
        if labels.count(predicate_index) == 0:
          labels.append(predicate_index)
      data['labels'].append(labels)
  return pd.DataFrame.from_dict(data), predicate_list

## 获取训练数据

In [42]:
train_data, predicate_list = prepare_data('/tmp/train_data.json')

sample {} 0
sample {} 10000
sample {} 20000
sample {} 30000
sample {} 40000
sample {} 50000
sample {} 60000
sample {} 70000
sample {} 80000
sample {} 90000
sample {} 100000
sample {} 110000
sample {} 120000
sample {} 130000
sample {} 140000
sample {} 150000
sample {} 160000
sample {} 170000


In [43]:
train_data

,text,labels
0,如何演好自己的角色，请读《演员自我修养》《喜剧之王》周星驰崛起于穷困潦倒之中的独门秘笈,[0]
1,茶树茶网蝽，Stephanitis chinensis Drake，属半翅目网蝽科冠网椿属的...,[1]
2,丝角蝗科，Oedipodidae，昆虫纲直翅目蝗总科的一个科,[1]
3,爱德华·尼科·埃尔南迪斯（1986-），是一位身高只有70公分哥伦比亚男子，体重10公斤，只...,"[2, 3, 4]"
4,《逐风行》是百度文学旗下纵横中文网签约作家清水秋风创作的一部东方玄幻小说，小说已于2014-...,"[5, 6]"
...,...,...
173103,《你是如此难以忘记》由周治平作词作曲，张智霖演唱，收录在其2014年发行的翻唱专辑《Déjà...,"[21, 22]"
173104,《报告老板之豪言壮旅》是由刘循子墨执导的一部剧情剧，白客、刘循子墨、小爱、张本煜等主演,"[0, 10]"
173105,孟珙再次担起重任，统领南宋两大战场（南宋四大战区：川蜀、荆襄（又称京湖）、淮西、淮东），率领...,[12]
173106,黄道周（1585-1646），字幼平、幼玄、细遵、螭若，号石斋、石道人，福建漳浦人，明代书画...,"[25, 9]"


# 步骤3


## 引入BERT

In [44]:
!pip install bert-tensorflow

import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

## 加载BERT hub

In [45]:
BERT_MODEL_HUB = 'https://tfhub.dev/google/bert_chinese_L-12_H-768_A-12/1'
TRAINABLE = True

def create_tokenizer_from_hub_module():
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature='tokenization_info', as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info['vocab_file'], tokenization_info['do_lower_case']])
    return bert.tokenization.FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [46]:
tokenizer.tokenize("《宝贝》张悬这首歌也是小暖自己从孕期就开始一直循环听的，俏皮好听")

['《',
 '宝',
 '贝',
 '》',
 '张',
 '悬',
 '这',
 '首',
 '歌',
 '也',
 '是',
 '小',
 '暖',
 '自',
 '己',
 '从',
 '孕',
 '期',
 '就',
 '开',
 '始',
 '一',
 '直',
 '循',
 '环',
 '听',
 '的',
 '，',
 '俏',
 '皮',
 '好',
 '听']

In [47]:
MAX_SEQ_LENGTH=256
predicate_list_map = [0]*len(predicate_list)
for (index, predicate) in enumerate(predicate_list):
  predicate_list_map[index] = index

train_input_examples = train_data.apply(lambda x: bert.run_classifier.InputExample(guid=None, text_a=x['text'], text_b=None, label=x['labels']), axis=1)
train_input_features = conver_examples_to_features(train_input_examples, predicate_list_map, MAX_SEQ_LENGTH, tokenizer)


INFO:tensorflow:Writing example 0 of 173108


INFO:tensorflow:Writing example 0 of 173108


INFO:tensorflow:Writing example 10000 of 173108


INFO:tensorflow:Writing example 10000 of 173108


INFO:tensorflow:Writing example 20000 of 173108


...

INFO:tensorflow:Writing example 150000 of 173108


INFO:tensorflow:Writing example 160000 of 173108


INFO:tensorflow:Writing example 160000 of 173108


INFO:tensorflow:Writing example 170000 of 173108


INFO:tensorflow:Writing example 170000 of 173108


In [48]:
train_input_features[0].label_ids

[1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

# 步骤4



## 创建model_fn_builder


In [0]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels, num_labels):
  bert_module = hub.Module(BERT_MODEL_HUB, trainable=False)
  bert_input = dict(input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids)
  bert_output = bert_module(inputs=bert_input, signature='tokens', as_dict=True)
  print(tf.get_variable_scope().name)

  output_layer = bert_output['pooled_output']
  hidden_size = output_layer.shape[-1].value
  print(input_ids)
  print([num_labels, hidden_size])
  print(num_labels)
 
  output_weights = tf.get_variable('output_weights', [num_labels, hidden_size], initializer=tf.truncated_normal_initializer(stddev=0.02))  
  output_bias = tf.get_variable('output_bias', [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope('loss'):
    print(output_weights)

    # if not is_predicting:
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)
    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.sigmoid(logits)
    label_ids = tf.cast(labels, tf.float32)
    per_example_loss = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=label_ids), axis=-1)
    loss = tf.reduce_mean(per_example_loss)

  return loss, per_example_loss, logits, log_probs
     

In [0]:
def model_fn_builder(num_labels, learning_rate, num_train_steps, num_warmup_steps):
  def model_fn(features, labels, mode, params):
    input_ids = features['input_ids']
    input_mask = features['input_mask']
    segment_ids = features['segment_ids']
    label_ids = features['label_ids']
    is_predicting = (mode==tf.estimator.ModeKeys.PREDICT)
    (loss, per_example_loss, logits, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

    if not is_predicting:
      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu = False)
      def metric_fn(loss, label_ids, log_probs, per_example_loss):
        predicted_ids = tf.cast(log_probs > 0.5, tf.int32)
        label_ids = tf.cast(label_ids, tf.int32)
        elements_equal = tf.cast(tf.equal(predicted_ids, label_ids), tf.int32)
        row_predict_ids = tf.reduce_sum(elements_equal, -1)
        row_label_ids = tf.reduce_sum(tf.ones_like(label_ids), -1)
        accuracy = tf.metrics.accuracy(labels=row_label_ids, predictions=row_predict_ids)
        loss = tf.metrics.mean(values=per_example_loss)
        return{
            'eval_accuracy': accuracy,
            'eval_loss': loss
        }
      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode = mode, loss = loss, train_op = train_op)
      else:
        eval_metrics = metric_fn(loss, label_ids, log_probs, per_example_loss)
        return tf.estimator.EstimatorSpec(mode = mode, loss = loss, eval_metric_ops=eval_metrics)
    else:
      predictions = {
          'logits': logits,
          'probabilities': log_probs,
      }
      return tf.estimator.EstimatorSpec(mode = mode, predictions=predictions)

  return model_fn


# 步骤5


## 训练

In [0]:
BATCH_SIZE=16
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
WARMUP_PROPORTION = 0.1
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [0]:
num_train_steps = int(len(train_input_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)


In [0]:
run_config = tf.estimator.RunConfig(
    model_dir = OUTPUT_DIR,
    save_summary_steps = SAVE_SUMMARY_STEPS,
    save_checkpoints_steps = SAVE_CHECKPOINTS_STEPS
)

In [54]:
model_fn = model_fn_builder(
    num_labels = len(predicate_list),
    learning_rate = LEARNING_RATE,
    num_train_steps = num_train_steps,
    num_warmup_steps = num_warmup_steps
)

estimator = tf.estimator.Estimator(
    model_fn = model_fn,
    config = run_config,
    params = {'batch_size': BATCH_SIZE}
)

INFO:tensorflow:Using config: {'_model_dir': 'gs://bert_classification/training_output1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f350a53def0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': 'gs://bert_classification/training_output1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f350a53def0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [0]:
train_input_fn = input_fn_builder(
    features = train_input_features,
    seq_length = MAX_SEQ_LENGTH,
    is_training = True,
    num_labels = len(predicate_list),
    drop_remainder = False
)

In [56]:
from datetime import datetime
print(f'Begining Training!')
current_time = datetime.now()
estimator.train(input_fn = train_input_fn, max_steps=num_train_steps)
print('Training took time', datetime.now() - current_time)


Begining Training!
INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore



Tensor("IteratorGetNext:0", shape=(?, 256), dtype=int32)
[49, 768]
49
<tf.Variable 'output_weights:0' shape=(49, 768) dtype=float32_ref>
INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving checkpoints for 32457 into gs://bert_classification/training_output1/model.ckpt.


INFO:tensorflow:Saving checkpoints for 32457 into gs://bert_classification/training_output1/model.ckpt.


INFO:tensorflow:Loss for final step: 2.1056855.


INFO:tensorflow:Loss for final step: 2.1056855.


Training took time 6:25:08.047764


# 步骤 6

## 测试

In [57]:
!head /tmp/dev_data.json

{"postag": [{"word": "查尔斯", "pos": "nr"}, {"word": "·", "pos": "w"}, {"word": "阿兰基斯", "pos": "nr"}, {"word": "（", "pos": "w"}, {"word": "Charles Aránguiz", "pos": "nz"}, {"word": "）", "pos": "w"}, {"word": "，", "pos": "w"}, {"word": "1989年4月17日", "pos": "t"}, {"word": "出生", "pos": "v"}, {"word": "于", "pos": "p"}, {"word": "智利圣地亚哥", "pos": "ns"}, {"word": "，", "pos": "w"}, {"word": "智利", "pos": "ns"}, {"word": "职业", "pos": "n"}, {"word": "足球", "pos": "n"}, {"word": "运动员", "pos": "n"}, {"word": "，", "pos": "w"}, {"word": "司职", "pos": "v"}, {"word": "中场", "pos": "n"}, {"word": "，", "pos": "w"}, {"word": "效力", "pos": "v"}, {"word": "于", "pos": "p"}, {"word": "德国", "pos": "ns"}, {"word": "足球", "pos": "n"}, {"word": "甲级", "pos": "a"}, {"word": "联赛", "pos": "n"}, {"word": "勒沃库森足球俱乐部", "pos": "nt"}], "text": "查尔斯·阿兰基斯（Charles Aránguiz），1989年4月17日出生于智利圣地亚哥，智利职业足球运动员，司职中场，效力于德国足球甲级联赛勒沃库森足球俱乐部", "spo_list": [{"predicate": "出生地", "object_type": "地点", "subject_type": "人物", "object": "圣地亚哥", "subjec

In [69]:

test_data, _ = prepare_data('/tmp/dev_data.json', is_training=False, predicate_list=predicate_list)
print(test_data)
test_input_examples = test_data.apply(lambda x: bert.run_classifier.InputExample(guid=None, text_a=x['text'], text_b=None, label=x['labels']), axis=1)
test_input_features = conver_examples_to_features(test_input_examples, predicate_list, MAX_SEQ_LENGTH, tokenizer)
test_input_fn = input_fn_builder(
    features = test_input_features,
    seq_length = MAX_SEQ_LENGTH,
    is_training = False,
    num_labels = len(predicate_list),
    drop_remainder = False)
estimator.evaluate(input_fn=test_input_fn, steps=None)

sample {} 0
sample {} 10000
sample {} 20000
                                                    text    labels
0      查尔斯·阿兰基斯（Charles Aránguiz），1989年4月17日出生于智利圣地亚哥...    [9, 3]
1                                          《离开》是由张宇谱曲，演唱   [7, 22]
2      《愤怒的唐僧》由北京吴意波影视文化工作室与优酷电视剧频道联合制作，故事以喜剧元素为主，讲述唐...  [18, 10]
3       李治即位后，萧淑妃受宠，王皇后为了排挤萧淑妃，答应李治让身在感业寺的武则天续起头发，重新纳入后宫  [13, 14]
4      《工业4.0》是2015年机械工业出版社出版的图书，作者是（德）阿尔冯斯·波特霍夫，恩斯特·...   [6, 20]
...                                                  ...       ...
21634                                    7气候云林县属南亚热带湿润气候      [11]
21635  《可持续建筑设计实践》是2006年中国建筑工业出版社出版的图书，作者是斯泰里奥斯·普莱尼奥斯，纪雁   [6, 20]
21636                    武汉土木石建筑装饰设计工程有限公司于2012年10月25日成立      [24]
21637                         《网游之最强时代》是创世中文网连载的小说，作者是江山    [5, 6]
21638                       《我们的少年时代》程砚秋在该剧中饰演富家千金，陶西前女友       [0]

[21639 rows x 2 columns]
INFO:tensorflow:Writing example 0 of 21639


INFO:tensorflow:Writing example 0 of 21639


INFO:tensorflow:Writing example 10000 of 21639


INFO:tensorflow:Writing example 10000 of 21639


INFO:tensorflow:Writing example 20000 of 21639


...

INFO:tensorflow:Saving dict for global step 32457: eval_accuracy = 0.33508942, eval_loss = 2.9516234, global_step = 32457, loss = 2.951437


INFO:tensorflow:Saving dict for global step 32457: eval_accuracy = 0.33508942, eval_loss = 2.9516234, global_step = 32457, loss = 2.951437


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 32457: gs://bert_classification/training_output1/model.ckpt-32457


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 32457: gs://bert_classification/training_output1/model.ckpt-32457


{'eval_accuracy': 0.33508942,
 'eval_loss': 2.9516234,
 'global_step': 32457,
 'loss': 2.951437}

# 步骤7

## 预测

In [0]:
def get_prediction(in_sentences):
  input_examples = [bert.run_classifier.InputExample(guid="", text_a = x,
                                                     text_b=None, label=[]) for x in in_sentences]

  input_features = conver_examples_to_features(input_examples, predicate_list_map,MAX_SEQ_LENGTH,
                                                tokenizer)
  predict_input_fn = input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH,
                                      is_training=False, num_labels=len(predicate_list), drop_remainder=False)
  predictions = estimator.predict(predict_input_fn)
  return [(sentence, prediction['probabilities'], prediction['logits']) 
          for sentence, prediction in zip(in_sentences, predictions)]
                                                  

In [162]:
predictions = get_prediction([
                             
                              # '《愤怒的唐僧》由北京吴意波影视文化工作室与优酷电视剧频道联合制作，故事以喜剧元素为主，讲述唐僧与佛祖打牌，得罪了佛祖，被踢下人间再渡九九八十一难的故事',
                              # '李治即位后，萧淑妃受宠，王皇后为了排挤萧淑妃，答应李治让身在感业寺的武则天续起头发，重新纳入后宫',
                              # '《狙击部队》是由江苏海润影视制作有限公司出品的电视剧，由赵浚凯执导，王珂、赵聪、刘鑫等主演'
                              # '2013年3月13日，福建广电网络集团厦门分公司举行成立大会，厦门广电网络已完成资产、人员、收入划转等各项整合工作，并正式挂牌运行，这标志着作为我省文化体制改革三项刚性任务之一的广电网络整合工作已全面完成'
# '蒲松龄在他70岁时完成文学作品《聊斋》'

        '《杨家将》由中国台湾导演顾辉雄执导，狄珊编剧，杨丽花、许秀年、小凤仙、柳青、吴翠娥、洪秀玉、潘丽丽、许仙姬主演的历史、传记类电视歌仔戏，《杨家将》是1982年的中国台湾拍摄的一部电视歌仔戏'
                              ])
            
# get_prediction([])

INFO:tensorflow:Writing example 0 of 1


INFO:tensorflow:Writing example 0 of 1


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Restoring parameters from gs://bert_classification/training_output1/model.ckpt-32457


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [163]:
predictions

[('《杨家将》由中国台湾导演顾辉雄执导，狄珊编剧，杨丽花、许秀年、小凤仙、柳青、吴翠娥、洪秀玉、潘丽丽、许仙姬主演的历史、传记类电视歌仔戏，《杨家将》是1982年的中国台湾拍摄的一部电视歌仔戏',
  array([6.2644058e-01, 1.2675554e-02, 2.3445189e-03, 2.4697393e-02,
         1.9636422e-02, 1.3033450e-03, 1.5876353e-02, 6.0574681e-02,
         1.5225410e-03, 2.9105425e-02, 5.5457872e-01, 6.5958500e-04,
         1.0185182e-02, 4.8187375e-03, 4.0018260e-03, 8.4395409e-03,
         4.3937862e-03, 7.1218073e-02, 1.7450145e-01, 7.6180995e-03,
         2.4155647e-02, 5.1687151e-02, 4.5142353e-02, 6.2690675e-03,
         4.2373836e-03, 7.9384148e-03, 4.6911538e-03, 3.3016413e-02,
         2.3778081e-03, 4.6046972e-03, 2.3342073e-03, 5.9536129e-02,
         2.0400286e-03, 9.6252561e-04, 4.8778057e-03, 8.1969500e-03,
         8.5475445e-03, 9.7143650e-04, 1.6528666e-03, 3.3423603e-03,
         4.7066808e-04, 2.5500357e-03, 9.3594193e-04, 2.2263527e-03,
         2.9259920e-04, 4.4003129e-04, 1.0243058e-04, 3.9756298e-05,
         5.9376591e-05], dtype=float32),
  array([  0.51697683,  -4.3553

In [165]:
labels = []
for text, probilities, logits in predictions:
  label=[]
  for index,prob in enumerate(probilities):
    if prob>0.5:
      label.append(predicate_list[index])
  labels.append(label)

labels

[['主演', '导演']]

In [63]:
in_sentences = [
                             
                              '《愤怒的唐僧》由北京吴意波影视文化工作室与优酷电视剧频道联合制作，故事以喜剧元素为主，讲述唐僧与佛祖打牌，得罪了佛祖，被踢下人间再渡九九八十一难的故事',
                              '李治即位后，萧淑妃受宠，王皇后为了排挤萧淑妃，答应李治让身在感业寺的武则天续起头发，重新纳入后宫',
                              ]

input_examples = [bert.run_classifier.InputExample(guid="", text_a = x,
                                            text_b=None, label=[]) for x in in_sentences]

input_features = conver_examples_to_features(input_examples, predicate_list_map,MAX_SEQ_LENGTH,
                                      tokenizer)
predict_input_fn = input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH,
                            is_training=False, num_labels=len(predicate_list), drop_remainder=False)
predictions = estimator.predict(predict_input_fn)


INFO:tensorflow:Writing example 0 of 2


INFO:tensorflow:Writing example 0 of 2


In [64]:
#[var for var in tf.global_variables() ]
estimator.get_variable_value('output_wegihts')

NotFoundError: ignored